# PyPSA-VGR

## Input

## The network

This model is created at a level of aggregated production and consumption in VGR of each type of energy production, storage, and consumption.

The network contains:
- A main bus with
    - A static target load (representing the minimal target load desired, see Optimization below)
    - An onshore wind generator
    - An offshore wind generator
    - A solar generator
- A battery bus with
    - A store (battery)
    - Two links to and from the main bus with efficiencies <1 and a capital cost representing the inverter
- A hydrogen bus with
    - 
- A market bus with
    -


## Optimization

In [ ]:
# Config and imports

import pypsa
import logging
import warnings
import atlite
import time
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.ops import unary_union
from shapely.geometry import Polygon
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import ipywidgets as widgets
from IPython.display import display
import io
import base64
import atlite
import xarray as xr
from rasterio.plot import show
from atlite.gis import shape_availability, ExclusionContainer
from matplotlib.gridspec import GridSpec

logging.basicConfig(level=logging.INFO)

## Weather data window
START="2023-01"
END="2023-12"
RESOLUTION = 3 #3h window for weather data and pypsa model optimization

## Västra götalands län
selected_lan_code = "14" 

## Single municipal
selected_kom_code = "1480" # Gbg

## Or all municipals
selected_kom_code = None

## Wind turbines
WIND_TURBINE = "Vestas_V25_200kW"
WIND_TURBINE = "Vestas_V47_660kW"
WIND_TURBINE = "Bonus_B1000_1000kW"
WIND_TURBINE = "Suzlon_S82_1.5_MW"
WIND_TURBINE = "Vestas_V66_1750kW"
WIND_TURBINE = "Vestas_V80_2MW_gridstreamer"
WIND_TURBINE = "Siemens_SWT_2300kW"
WIND_TURBINE = "Vestas_V90_3MW"
WIND_TURBINE_OFFSHORE = "Vestas_V90_3MW"

In [ ]:
# Create main cutout for VGR/municipality

sweden = gpd.read_file("data/geo/georef-sweden-kommun@public.geojson") #Source Lantmäteriverket, data maintained by opendatasoft.com

vgr = sweden.loc[sweden.lan_code.apply(lambda x: selected_lan_code in x)]

minx, miny, maxx, maxy = vgr.total_bounds

cutout = atlite.Cutout(
    path=str(f"vgr-{selected_kom_code}-{START}-{END}.nc"),
    module="era5",
    x=slice(minx, maxx),
    y=slice(miny, maxy),
    time=slice(START,END),
    dx=0.125,
    dy=0.125,
    dt="3h"
)

start_time = time.time()

cutout.prepare(features=['influx', 'temperature', 'wind'])

print("Execution time: %.4f minutes" % ((time.time() - start_time) / 60))


if selected_kom_code is None:
    selection = gpd.GeoDataFrame(geometry=[unary_union(vgr.geometry)], crs=vgr.crs)
else:
    selection = vgr.loc[vgr['kom_code'].isin(selected_kom_code)]

# EEZ (Economical zone)
shapefile_path = "data/geo/Ekonomiska_zonens_yttre_avgränsningslinjer/Ekonomiska_zonens_yttre_avgränsningslinjer_linje.shp"
eez_shape = gpd.read_file(shapefile_path).to_crs(selection.crs)
min_x, min_y, max_x, max_y = eez_shape.total_bounds
# Arbitrarily using min/max from cutout or eez to visualize it on VGR region
bounding_box = Polygon([(min_x, miny), (min_x, maxy), (maxx, maxy), (maxx, miny)])
bounds = gpd.GeoDataFrame(geometry=[bounding_box], crs=selection.crs) 
eez = gpd.overlay(eez_shape, bounds, how='intersection')

In [ ]:
# Exclude land use per solar/wind

# Source: https://www.uts.edu.au/oecm/renewable-resource-mapping
# (Classification mapping: https://collections.sentinel-hub.com/corine-land-cover/readme.html)
# With the exception of "41; Water bodies"

# Land-cover classes NOT included for solar areas:
EXCLUDED_SOLAR = [1,4,5,6,7,9,10,11,23,24,25,27,30,33,34,35,36,37,38,39,40,41,42,43,44] # 1.1.1,1.2.2,1.2.3,1.2.4,1.3.1,1.3.3,1.4.1,1.4.2,3.1.1,3.1.2,3.1.3,3.2.2,3.3.1,3.3.4,3.3.5,4.1.1,4.1.2,4.2.1,4.2.2,4.2.3,5.1.1,5.1.2,5.2.1,5.2.2,5.2.3
# Land-cover classes NOT included for wind energy areas:
EXCLUDED_WIND_NON_OCEAN = [1,2,3,4,5,6,7,9,10,11,15,16,17,23,24,25,27,30,31,33,34,35,36,37,38,39,40,41,42,43,44] # 1.1.1,1.1.2,1.2.1,1.2.2,1.2.3,1.2.4,1.3.1,1.3.3,1.4.1,1.4.2,2.2.1,2.2.2,2.2.3,3.1.1,3.1.2,3.1.3,3.2.2,3.3.1,3.3.2,3.3.4,3.3.5,4.1.1,4.1.2,4.2.1,4.2.2,4.2.3,5.1.1,5.1.2,5.2.1,5.2.2,5.2.3
INCLUDED_WIND_OCEAN = [44, 41] # 5.2.3, 5.1.2


CORINE = "data/geo/corine.tif"
solar_excluder = ExclusionContainer()
onwind_excluder = ExclusionContainer()
offwind_excluder = ExclusionContainer()

solar_excluder.add_raster(CORINE, codes=EXCLUDED_SOLAR)
onwind_excluder.add_raster(CORINE, codes=EXCLUDED_WIND_NON_OCEAN)
offwind_excluder.add_raster(CORINE, codes=INCLUDED_WIND_OCEAN, invert=True)

solar_avail = cutout.availabilitymatrix(selection, solar_excluder)
onwind_avail = cutout.availabilitymatrix(selection, onwind_excluder)
offwind_avail = cutout.availabilitymatrix(selection, offwind_excluder)

In [ ]:
# Atlite availability results : SOLAR

solar_availability_matrix = solar_avail.stack(spatial=["y", "x"])
mean_solar_capacity_factor = cutout.pv(
    matrix=solar_availability_matrix,
    panel=atlite.solarpanels.CdTe,
    orientation="latitude_optimal",
    index=selection.index,
    per_unit =True,
)

In [ ]:
# Atlite availability results : WIND ONSHORE

onwind_availability_matrix = onwind_avail.stack(spatial=["y", "x"])
mean_onwind_capacity_factor = cutout.wind(
    matrix=onwind_availability_matrix,
    turbine = f"{WIND_TURBINE}",
    index=selection.index,
    per_unit =True,
)

mean_onwind_capacity_factor.values.flatten().sum()

In [ ]:
# Atlite availability results : WIND OFFSHORE

offwind_availability_matrix = offwind_avail.stack(spatial=["y", "x"])
mean_offwind_capacity_factor = cutout.wind(
    matrix=offwind_availability_matrix,
    turbine = f"{WIND_TURBINE_OFFSHORE}",
    index=selection.index,
    per_unit =True,
)

mean_offwind_capacity_factor.values.flatten().sum()

In [ ]:
# Load demand data from kontrollrummet.se (using hourly data from 2023)

## Create index
index = pd.to_datetime(cutout.coords['time'])[:2920]

## Load the data, use only the timestamp column and the total consumption for SE3 (in column 4) and skip the first 5 rows that consist of header data. Create a new header.
path_to_data = 'data/demand/timvarden-2023-01-12.csv'
timvarden = pd.read_csv(path_to_data, delimiter=',', skiprows=5, usecols=[0,3], header=None)
timvarden.columns = ['timestamp', 'se3']

## Create an index from the timestamp column. Convert the se3 column to float, remove the thousand separator and change the sign to positive.
timvarden['timestamp'] = pd.to_datetime(timvarden['timestamp'], format='%d.%m.%Y %H:%M')
timvarden.set_index('timestamp', inplace=True)
timvarden['se3'] = -timvarden['se3'].str.replace(',', '').astype(float)

## Resample the hourly data to 3h data selecting the max value in each 3h period
tretimvarden = timvarden.resample('3h').max()

## Normalize the load so that total production for the year is 1MWh
normalized_load = tretimvarden / (tretimvarden.sum() * 3)

## VGR load is the normalized load pattern producing 19 TWh (19,000,000 MWh) per year
vgr_load = normalized_load * 19_000_000

In [ ]:
# Parameters for iteration

## Set the target load
target_load = vgr_load.values.flatten()

## Biogas production parameters
biogas_production_max_nominal = 1000 # MW

## Set bus colors for dashboard
bus_colors = ["black", "yellow", "blue", "blue", "green", "red", "red", "red"]


In [ ]:
# Load cost parameters

parameters = pd.read_csv("data/assumptions.csv")
parameters.set_index(['technology', 'parameter'], inplace=True)

In [ ]:
# Build the network

## Initialize the network
network = pypsa.Network()
network.set_snapshots(index)
network.snapshot_weightings.loc[:, :] = RESOLUTION

## Carriers
carriers = [
    'AC',
    'onwind',
    'offwind',
    'solar',
    'li-ion',
    'h2',
    'biogas',
    ]

carrier_colors = ['black', 'green', 'blue', 'red', 'lightblue', 'grey', 'brown']

network.madd(
    'Carrier',
    carriers,
    color=carrier_colors,
    )

## Main bus location
midx = (minx + maxx)/2
midy = (miny + maxy)/2

## Add the buses
network.add('Bus', 'Main bus', carrier='AC', x=midx, y=midy)
network.add('Bus', 'Solar', x=midx+0.5, y=midy+0.25)
network.add('Bus', 'Onwind', x=midx+0.5, y=midy-0.15)
network.add('Bus', 'Offwind', x=midx-1.25, y=midy-0.75)
network.add('Bus', 'Battery', carrier='li-ion', x=midx-0.5, y=midy)
network.add('Bus', 'Gas turbine', x=midx, y=midy+0.5)
network.add('Bus', 'H2 storage', carrier='h2', x=midx-0.5, y=midy+0.5)
network.add('Bus', 'Biogas market', x=midx, y=midy+0.9)

## Add loads
network.add('Load', 'Desired load', bus='Main bus',
            p_set=target_load
            )

network.add('Generator', 'Backstop', carrier='AC', bus='Main bus',
            p_nom_extendable=False,
            p_set=0,
            capital_cost=parameters.loc[('backstop', 'capital_cost'), 'value'],
            marginal_cost=parameters.loc[('backstop', 'marginal_cost'), 'value'],
            lifetime=parameters.loc[('backstop', 'lifetime'), 'value'],
            )

## Add generators

### Solar
network.add('Generator', 'Solar park', carrier='solar', bus='Solar',
            p_nom_extendable=True, 
            p_max_pu=mean_solar_capacity_factor.values.flatten(),
            capital_cost=parameters.loc[('solar', 'capital_cost'), 'value'],
            #marginal_cost=assumptions.loc['solar-utility','FOM'].value / 100 * assumptions.loc['solar-utility','investment'].value / (mean_solar_capacity_factor.values.flatten().sum() * 3),
            lifetime=parameters.loc[('solar', 'lifetime'), 'value'],
            )

network.add('Link', 'Solar link',
            bus0='Solar',
            bus1='Main bus',
            p_nom_extendable=True,
            )

### Onwind
network.add('Generator', 'Onwind park', carrier='onwind', bus='Onwind',
            p_nom_extendable=True,
            p_max_pu=mean_onwind_capacity_factor.values.flatten(),
            capital_cost=parameters.loc[('onwind', 'capital_cost'), 'value'],
            #marginal_cost=assumptions.loc['onwind','VOM'].value + assumptions.loc['onwind','FOM'].value / 100 * assumptions.loc['onwind','investment'].value  / (mean_onwind_capacity_factor.values.flatten().sum() * 3),
            lifetime=parameters.loc['onwind','lifetime'].value,
            )

network.add('Link', 'Onwind link',
            bus0='Onwind',
            bus1='Main bus',
            p_nom_extendable=True,
            )

### Offwind
network.add('Generator', 'Offwind park', carrier='offwind', bus='Offwind',
            p_nom_extendable=True, 
            p_max_pu=mean_offwind_capacity_factor.values.flatten(),
            capital_cost=parameters.loc[('offwind', 'capital_cost'), 'value'],
            #marginal_cost=assumptions.loc['onwind','VOM'].value + assumptions.loc['offwind','FOM'].value / 100 * assumptions.loc['offwind','investment'].value  / (mean_offwind_capacity_factor.values.flatten().sum() * 3),
            lifetime=parameters.loc['offwind','lifetime'].value,
            )

network.add('Link', 'Offwind link',
            bus0='Offwind',
            bus1='Main bus',
            p_nom_extendable=True,
            )

## Add H2 electrolysis, storage, pipline to gas turbine

network.add('Link',
            'H2 electrolysis',
            bus0='Main bus',
            bus1='H2 storage',
            p_nom_extendable=True,
            efficiency=parameters.loc['h2_electrolysis','efficiency'].value,
            capital_cost=parameters.loc['h2_electrolysis','capital_cost'].value,
            #marginal_cost=230+55,
            lifetime=parameters.loc['h2_electrolysis','lifetime'].value,
            )

network.add('Store', 'H2 storage', carrier='h2', bus='H2 storage',
            e_nom_extendable=True,
            e_initial=150_000,
            # e_nom_max=target_load.max()*hours_h2_storage,
            capital_cost=parameters.loc['h2_storage','capital_cost'].value,
            lifetime=parameters.loc['h2_storage','lifetime'].value
            )

network.add('Link',
            'H2 pipeline',
            bus0='H2 storage',
            bus1='Gas turbine',
            p_nom_extendable=True,
            efficiency=parameters.loc['h2','efficiency'].value,
            )

### Biogas pipeline

network.add('Generator', 'Biogas market', carrier='biogas', bus='Biogas market',
            p_nom_extendable=True,
            p_nom_max=biogas_production_max_nominal,
            marginal_cost=parameters.loc['biogas','cost'].value,
            lifetime=100,
            )

network.add('Link', 'Biogas pipeline', bus0='Biogas market', bus1='Gas turbine',
            p_nom_extendable=True,
            efficiency=parameters.loc['biogas','efficiency'].value,
            )

### Gas turbine
network.add('Link', 'Gas turbine', bus0='Gas turbine', bus1='Main bus',
            p_nom_extendable=True,
            capital_cost=parameters.loc['gas_turbine','capital_cost'].value,
            #marginal_cost=assumptions.loc['CCGT','VOM'].value,
            lifetime=parameters.loc['gas_turbine','lifetime'].value,
            )

## Add battery storage TODO: add running cost, 
network.add('Store', 'Battery', carrier='li-ion', bus='Battery',
            e_nom_extendable=True,
            e_cyclic=True,
            e_min_pu=0.15,
            standing_loss=0.00008, # TODO: Check if this is really per hour as in the documentation or if it is per snapshot
            capital_cost=parameters.loc['battery_storage', 'capital_cost'].value,
            lifetime=parameters.loc['battery_storage', 'lifetime'].value,
            )

network.add('Link','Battery charge',
            bus0 = 'Main bus',
            bus1 = 'Battery',
            efficiency = parameters.loc['battery_inverter','efficiency'].value,
            p_nom_extendable = True,
            capital_cost=parameters.loc['battery_inverter','capital_cost'].value,
            #marginal_cost=0,#assumptions.loc['battery inverter','FOM'].value / 100 * assumptions.loc['battery inverter','investment'].value,
            lifetime=parameters.loc['battery_inverter','lifetime'].value,
            )

network.add('Link','Battery discharge',
            bus0 = 'Battery',
            bus1 = 'Main bus',
            efficiency = parameters.loc['battery_inverter','efficiency'].value,
            p_nom_extendable = True,
            )

## Nuclear


In [ ]:
# Run the optimization

model = network.optimize.create_model()

generator_capacity = model.variables["Generator-p_nom"]
link_capacity = model.variables["Link-p_nom"]
link_flow = model.variables["Link-p"]
store_level = model.variables["Store-e"]


## Solar link capacity
solar_link_constraint = generator_capacity.loc['Solar park'] - link_capacity.loc['Solar link']
model.add_constraints(solar_link_constraint == 0, name="Solar_park-solar_link-match")

## Onwind link capacity
onwind_link_constraint = generator_capacity.loc['Onwind park'] - link_capacity.loc['Onwind link']
model.add_constraints(onwind_link_constraint == 0, name="Onwind_park-onwind_link-match")

## Offwind link capacity
offwind_link_constraint = generator_capacity.loc['Offwind park'] - link_capacity.loc['Offwind link']
model.add_constraints(offwind_link_constraint == 0, name="Offwind_park-offwind_link-match")

## H2 input + H2 store >= H2 output
#h2_flow_constraint = link_flow.loc["H2 electrolysis"] + store_level.loc['H2 storage'] - link_flow.loc["H2 pipeline"]
#model.add_constraints(h2_flow_constraint >= 0, name="H2_flow_constraint")

## Battery charge/discharge ratio
lhs = link_capacity.loc["Battery charge"] - network.links.at["Battery charge", "efficiency"] * link_capacity.loc["Battery discharge"]
model.add_constraints(lhs == 0, name="Link-battery_fix_ratio")

network.optimize.solve_model(solver_name='highs')


In [ ]:
# Plot the dashboard

## Parameters
window_size=56 # Rolling average window size for charts

## Create solar availability map widget
solar_map = widgets.Output()
with solar_map:
    solar_excluder.plot_shape_availability(selection, show_kwargs = { "cmap": "Reds" })
    plt.show()

## Create onwind availability map widget
onwind_map = widgets.Output()
with onwind_map:
    onwind_excluder.plot_shape_availability(selection, show_kwargs = { "cmap": "Greens" })
    plt.show()

## Create offwind availability map widget
offwind_map = widgets.Output()
with offwind_map:   
    offwind_excluder.plot_shape_availability(selection, show_kwargs = { "cmap": "Blues" })
    plt.show()

## Create solar capacity factor line chart widget
solar_capacity_line_chart = widgets.Output()
with solar_capacity_line_chart:
    fig, ax = plt.subplots(figsize=(12, 4))
    mean_solar_capacity_factor.plot(ax=ax, color="Red", alpha=0.2)
    mean_solar_capacity_factor.rolling(time=window_size, center=True).mean().plot(ax=ax, color="Red")
    ax.set_ylabel("Capacity factor")
    ax.set_xlabel('')
    plt.show()

## Create onwind capacity factor line chart widget
onwind_capacity_line_chart = widgets.Output()
with onwind_capacity_line_chart:
    fig, ax = plt.subplots(figsize=(12, 4))
    mean_onwind_capacity_factor.plot(ax=ax, color="Green", alpha=0.2)
    mean_onwind_capacity_factor.rolling(time=window_size, center=True).mean().plot(ax=ax, color="Green")
    ax.set_ylabel("Capacity factor")
    ax.set_xlabel('')
    plt.show()

## Create offwind capacity factor line chart widget
offwind_capacity_line_chart = widgets.Output()
with offwind_capacity_line_chart:
    fig, ax = plt.subplots(figsize=(12, 4))
    mean_offwind_capacity_factor.plot(ax=ax, color="Blue", alpha=0.2)
    mean_offwind_capacity_factor.rolling(time=window_size, center=True).mean().plot(ax=ax, color="Blue")
    ax.set_ylabel("Capacity factor")
    ax.set_xlabel('')
    plt.show()

## Create target load line chart widget
load_line_chart = widgets.Output()
with load_line_chart:
    fig, ax = plt.subplots(figsize=(14, 7))
    vgr_load.plot(ax=ax, color='grey', alpha=0.2)
    vgr_load.rolling(window=window_size, center=True).mean().plot(ax=ax, color='blue', alpha=1)
    ax.legend(['Demand (3h average)', 'Demand (weekly average'])
    ax.set_ylabel("Load [MW]")
    ax.set_xlabel('')
    plt.show()

## Create network map widget
warnings.filterwarnings("ignore", category=UserWarning, module="cartopy.mpl.feature_artist")
network_map = widgets.Output()
with network_map:
    fig, ax = plt.subplots(figsize=(12, 12), subplot_kw={'projection': ccrs.PlateCarree()})
    network.plot(ax=ax, boundaries = [minx-1, maxx+3, miny-1.5, maxy+4.5], color_geomap=True, bus_sizes=0.02, bus_colors=bus_colors)
    legend_labels = [plt.Line2D([0], [0], marker='o', color='w', label=bus, 
                            markerfacecolor=color, markersize=10) for bus, color in zip(network.buses.index, bus_colors)]
    plt.legend(handles=legend_labels, loc='upper left', bbox_to_anchor=(0.6, 0.7), ncol=1)
    plt.show()

## Format and create the parameters table widget
parameters_table = widgets.HTML(value=parameters.to_html())

## Format and create the system table widget widget
generator_column_names = {
    'capital_cost': 'Capital Cost',
    'marginal_cost': 'Marginal Cost',
    'lifetime': 'Lifetime',
    'p_nom_opt': 'Optimal Capacity'
}

store_column_names = {
    'capital_cost': 'Capital Cost',
    'marginal_cost': 'Marginal Cost',
    'lifetime': 'Lifetime',
    'e_nom_opt': 'Optimal Capacity'
}

generators = pd.concat([
    network.generators.loc[['Solar park', 'Onwind park', 'Offwind park']][['capital_cost', 'marginal_cost', 'lifetime', 'p_nom_opt']],
    network.links.loc[['Gas turbine']][['capital_cost', 'marginal_cost', 'lifetime', 'p_nom_opt']]
])

generators['capital_cost'] = generators['capital_cost'].apply(lambda x: f"{round(float(x)/1_000_000, 1)} MSEK/MW")
generators['marginal_cost'] = generators['marginal_cost'].apply(lambda x: f"{round(float(x), 1)} SEK/MWh")
generators['lifetime'] = generators['lifetime'].apply(lambda x: f"{int(round(float(x), 0))} years")
generators['p_nom_opt'] = generators['p_nom_opt'].apply(lambda x: f"{int(round(float(x), 0))} MW")
generators.rename(columns=generator_column_names, inplace=True)

stores = network.stores.loc[["H2 storage", "Battery"]][['capital_cost', 'marginal_cost', 'lifetime', 'e_nom_opt']]
stores.loc[:, 'capital_cost'] = stores['capital_cost'].apply(lambda x: f"{round(float(x)/1_000_000,1)} MSEK/MWh")
stores.loc[:, 'marginal_cost'] = stores['marginal_cost'].apply(lambda x: f"{round(float(x),1)} SEK/MWh")
stores.loc[:, 'lifetime'] = stores['lifetime'].apply(lambda x: f"{int(round(float(x), 0))} years")
stores.loc[:, 'e_nom_opt'] = stores['e_nom_opt'].apply(lambda x: f"{int(round(float(x), 0))} MWh")
stores.rename(columns=store_column_names, inplace=True)

system_table = widgets.HTML(value=pd.concat([generators, stores]).to_html())

## Format and create the stores table widget widget

## Create generators size pie chart widget
generator_pie_chart = widgets.Output()
with generator_pie_chart:
    network.generators['p_nom_opt'].plot(figsize=(4, 4), kind='pie', labels=None, legend=True, startangle=90, title='Generator sizes')
    plt.show()

## Create stores size pie chart widget
store_pie_chart = widgets.Output()
with store_pie_chart:
    network.stores.loc[["H2 storage", "Battery"]]['e_nom_opt'].plot(figsize=(4, 4), kind='pie', labels=None, legend=True, startangle=90, title='Energy store sizes')
    plt.axis('off')  # Turns off the axis
    plt.show()

## Create price line chart widget
price_line_chart = widgets.Output()
with price_line_chart:
    network.buses_t.marginal_price.plot(figsize=(24, 6), legend=True, title='Marginal price')
    plt.show()

## Create generators output line chart widget
generator_line_chart = widgets.Output()
with generator_line_chart:
    fig, ax = plt.subplots(figsize=(16,6))
    network.generators_t.p.abs().plot(ax=ax, kind='area', stacked=True, legend=True, alpha=0.2, title='Power output')
    network.generators_t.p.abs().rolling(window=window_size, center=True).mean().plot(ax=ax, kind='area', stacked=True)
    plt.show()

## Create generators total output pie chart
generator_pie_chart = widgets.Output()
with generator_pie_chart:
    network.generators_t.p.sum().plot(figsize=(8, 6), kind='pie', labels=None, legend=True, title='Power output')
    plt.show()

## Create stores line chart widget
store_line_chart = widgets.Output()
with store_line_chart:
    network.stores_t.e[['H2 storage', 'Battery']].plot(figsize=(24, 6),legend=True, title='Energy stored')
    plt.show()

## Build the dashboard and show

sun_wind_map = widgets.HBox([solar_map, onwind_map, offwind_map])
sun_wind_cfs = widgets.VBox([solar_capacity_line_chart, onwind_capacity_line_chart, offwind_capacity_line_chart])

row_1 = widgets.HBox([widgets.VBox([load_line_chart, sun_wind_map]), sun_wind_cfs])

row_2 = widgets.HBox([widgets.VBox([network_map, system_table, widgets.HBox([generator_pie_chart, store_pie_chart])]), parameters_table])

row_3 = widgets.HBox([generator_line_chart, generator_pie_chart])

dashboard = widgets.VBox([row_1, row_2, row_3, store_line_chart, price_line_chart])

dashboard




In [ ]:
fix, ax1 = plt.subplots(figsize=(24, 6))
                        
network.buses_t.marginal_price[['Main bus']].plot(ax=ax1, legend=True, title='Marginal price')

ax2 = plt.twinx(ax1)
network.stores_t.e[['H2 storage']].plot(ax=ax2,legend=True, color='red', title='Energy stored')



In [ ]:
fix, ax1 = plt.subplots(figsize=(24, 6))
                        
network.buses_t.marginal_price[['Main bus']].plot(ax=ax1, legend=True, title='Marginal price')

ax2 = plt.twinx(ax1)
vgr_load.plot(ax=ax2, color='grey', alpha=0.2)


In [ ]:
total_cost = network.statistics.capex().sum()
total_energy = network.generators_t.p[['Solar park', 'Onwind park', 'Offwind park', 'Biogas market']].sum().sum()

total_cost/total_energy

In [ ]:
total_cost = network.statistics.capex().sum()/30
total_energy = network.generators_t.p[['Solar park', 'Onwind park', 'Offwind park', 'Biogas market']].sum().sum()

total_cost/total_energy

In [ ]:
network.statistics.capex()

In [ ]:
4.3 * 5591

In [ ]:
network.statistics.capex()

In [ ]:
network.generators_t.p.abs().rolling(window=56, center=True).mean().plot(figsize=(16,6), kind='area', stacked=True)